# Training and Evaluation

We will take a first-pass at evaluating or technique to start understanding its efficacy. We will existing CNN architectures and evaluate its performance on our interested categories with and without using our interested categories.

In [1]:
import cv2
import datetime
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
# NOTE: Copied from clustering NB
def load_metadata(filename):
    with open(filename, 'r') as f:
        return [x.strip().split('\t') for x in f.readlines()]

In [3]:
def get_mobilenet(input_shape=(64,64,3)):
    return tf.keras.Sequential([
        tf.keras.applications.MobileNet(input_shape=input_shape, weights=None, include_top=False),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(200, activation='softmax'),
    ])

In [ ]:
def get_simplecnn(input_shape=(64,64,3)):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(512, (3,3), (1,1), input_shape=input_shape, activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(512, (2,2), (1,1), activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(256, (2,2), (1,1), activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(256, (2,2), (1,1), activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(256),
        tf.keras.layers.Dense(200),
    ])

In [ ]:
# simplecnn = get_simplecnn()
# simplecnn.summary()

In [4]:
mobilenet = get_mobilenet()
mobilenet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_64 (Model)    (None, 2, 2, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               131200    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               1

In [5]:
def load_image_data(metadata, input_size=(64,64)):
    return np.array([cv2.resize(cv2.imread(x[0]), input_size) / 255. for x in metadata])

In [6]:
def load_labels(metadata):
    labels = np.array([x[1] for x in metadata])
    distinct_labels = np.array([[x] for x in set(labels)])
    encoder = OneHotEncoder(sparse=True)
    encoder.fit(distinct_labels)
    y_train = encoder.transform([[x] for x in labels])
    y_train = np.array([x[1] for x in np.argwhere(y_train == 1)])
    return (y_train, encoder)

In [7]:
# TODO: Remove hardcoding
print('Loading data into memory...')
train_metadata = load_metadata('./metadata_output/train_metadata.txt')
X_train = load_image_data(train_metadata)
(y_train, encoder) = load_labels(train_metadata)

# Interested indices for test data filtering
interested_categories = ['n01882714', 'n04562935']
interested_one_hot = encoder.transform([[x] for x in interested_categories])
interested_indices = np.array([x[1] for x in np.argwhere(interested_one_hot == 1)])
print('Done.')

Loading data into memory...
Done.


In [8]:
# Encoding sanity checks;
assert(len(train_metadata) == len(y_train))
assert(len(set(y_train)) == 200)

In [9]:
# TODO: Shouldn't be hardcoded.
TRAIN_STEPS=5211
TEST_STEPS=500
NUM_EPOCHS=250
TEST_TRAIN_SPLIT=0.33
def train_model(model, X_train, y_train, name):    
    # Compile model                                                                                                      
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=3e-3),                                                           
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),                                  
                  metrics=['accuracy'])      
    
    # Stop early if we're not making good progress                                                                           
    early_stop_monitor = tf.keras.callbacks.EarlyStopping(                                                                   
        monitor='val_loss',                                                                                              
        restore_best_weights=True,                                                                                       
        patience=10)   

    # Prepare for checkpoints            
    checkpoint_path = './checkpoints/' + name + '/cp-{epoch:04d}.ckpt'                                   
    cp_callback = tf.keras.callbacks.ModelCheckpoint(                                                                    
        filepath=checkpoint_path,                                                                                    
        verbose=1,                                                                                                   
        save_weights_only=False,                                                                                     
        save_freq=25000000)

    # Tensorboard                                                                                                        
    log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")                                              
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    history = model.fit(                                                                                                 
        x=X_train,
        y=y_train,
        epochs=NUM_EPOCHS,                                                                                                  
        steps_per_epoch=TRAIN_STEPS,                                                                        
        validation_steps=TEST_STEPS,                                                                     
        callbacks=[tensorboard_callback, cp_callback, early_stop_monitor],                                           
        validation_split=TEST_TRAIN_SPLIT,                                                                             
        shuffle=True)

    return history

In [10]:
# Evaluate model on interesting inputs
def evaluate_model(model, test_sets):
    for test_set in test_sets:
        X = test_set[0]
        y = test_set[1]
        model.evaluate(X, y)

In [11]:
# Train and save model
train_model(mobilenet, X_train, y_train, 'mobilenet_imbalanced')
mobilenet.save("/models/{}".format('mobilenet_imbalanced'))
print('model saved')

Train on 46899 samples, validate on 23101 samples
Epoch 1/250
14112/46899 [========>.....................] - ETA: 2:27 - loss: 5.3023 - accuracy: 0.0045

KeyboardInterrupt: 

In [ ]:
# TODO: Load test data, filter for interest, evaluate model